<a href="https://colab.research.google.com/github/vmjs1234/TIMEGPT-TABULA-RDL/blob/main/Tabular/zero_shot_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-Shot Inference with Tabula Model
In this notebook, we demonstrate how to perform zero-shot inference using a Tabula model.

In [1]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the Tabula model and tokenizer
model_name = 'mlfoundations/tabula-8b'  # Replace with actual model name if different
# Use AutoModelForCausalLM for Llama-based models like Tabula
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [2]:
# Prepare tabular data for inference
# Example row of tabular data
row = {'age': 35, 'income': 50000, 'education': "Bachelor's"}  # Replace with your data

# Convert the row into a natural language prompt
prompt = f"The person's age is {row['age']}, their income is {row['income']}, and their education level is {row['education']}."

# Display the generated prompt
print(f'Generated Prompt: {prompt}')


Generated Prompt: The person's age is 35, their income is 50000, and their education level is Bachelor's.


In [3]:
from transformers import GenerationConfig

In [4]:
import signal
import os

class TimeoutException(Exception):
    pass


def timeout_handler(signum, frame):
    raise TimeoutException("Inference timed out")


# Perform zero-shot inference
def run_inference(prompt, timeout_seconds=300):  # Set timeout to 5 minutes (300 seconds)
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt')

    # Define generation configuration for faster inference
    generation_config = GenerationConfig(
        max_new_tokens=50,  # Limit the number of generated tokens
        num_beams=1,  # Use greedy decoding for faster inference
        do_sample=False,
    )

    # Set the timeout handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)

    try:
        # Generate predictions (inference) with generation configuration
        outputs = model.generate(**inputs, generation_config=generation_config)

        # Decode the output to get the prediction
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Display the prediction
        print(f'Prediction: {prediction}')

    except TimeoutException:
        print("Inference timed out. Returning partial results if available.")
        # You can add logic here to handle partial results if possible

    finally:
        signal.alarm(0)  # Disable the alarm

In [5]:
# prompt: Infer something from the model using some sample promt

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GenerationConfig
import signal
import os

# Load the Tabula model and tokenizer
model_name = 'mlfoundations/tabula-8b'  # Replace with actual model name if different
# Use AutoModelForCausalLM for Llama-based models like Tabula
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Inference timed out")

# Perform zero-shot inference
def run_inference(prompt, timeout_seconds=300):  # Set timeout to 5 minutes (300 seconds)
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt')

    # Define generation configuration for faster inference
    generation_config = GenerationConfig(
        max_new_tokens=50,  # Limit the number of generated tokens
        num_beams=1,  # Use greedy decoding for faster inference
        do_sample=False,
    )

    # Set the timeout handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)

    try:
        # Generate predictions (inference) with generation configuration
        outputs = model.generate(**inputs, generation_config=generation_config)

        # Decode the output to get the prediction
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return prediction

    except TimeoutException:
        print("Inference timed out. Returning empty string.")
        return ""

    finally:
        signal.alarm(0)  # Disable the alarm

# Example usage with a sample prompt
sample_prompt = "The quick brown fox jumps over the lazy dog."
prediction = run_inference(sample_prompt)
print(f"Prediction for '{sample_prompt}': {prediction}")


row = {'age': 25, 'income': 60000, 'education': "Master's"}
prompt = f"The person's age is {row['age']}, their income is {row['income']}, and their education level is {row['education']}."
prediction = run_inference(prompt)
print(f"Prediction for the tabular data: {prediction}")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prediction for 'The quick brown fox jumps over the lazy dog.': The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog.
Prediction for the tabular data: The person's age is 25, their income is 60000, and their education level is Master's. They have 1 dependent and their credit score is 700. They have a car loan with a balance of 15000 and a monthly payment of 500. They have a mortgage with a balance of 200000 and a monthly payment of 


In [10]:
# prompt: RUn inference on a sample data lkike labeled_examples = pd.DataFrame(
#     [
#         {"location": "New York", "temperature": 22, "humidity": 65, "wind_speed": 12, "pressure": 1012, "month": "July",
#          "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
#         {"location": "Los Angeles", "temperature": 26, "humidity": 60, "wind_speed": 7, "pressure": 1015,
#          "month": "July", "weather_yesterday": "Partly Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
#         {"location": "Chicago", "temperature": 18, "humidity": 70, "wind_speed": 15, "pressure": 1008, "month": "July",
#          "weather_yesterday": "Partly Cloudy", "precipitation": 0.1, "visibility": 8, "weather_today": "Cloudy"},
#         {
# target_example = pd.DataFrame(
#     [
#         {"location": "San Jose", "temperature": 23, "humidity": 55, "wind_speed": 8, "pressure": 1013,
# output = inference_model.predict(
#     target_example=target_example,

import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import signal

# Load the Tabula model and tokenizer
model_name = 'mlfoundations/tabula-8b'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Inference timed out")

def run_inference(prompt, timeout_seconds=300):
    inputs = tokenizer(prompt, return_tensors='pt')
    generation_config = GenerationConfig(
        max_new_tokens=50,
        num_beams=1,
        do_sample=False,
    )
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)
    try:
        outputs = model.generate(**inputs, generation_config=generation_config)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return prediction
    except TimeoutException:
        print("Inference timed out. Returning empty string.")
        return ""
    finally:
        signal.alarm(0)

labeled_examples = pd.DataFrame(
    [
        {"location": "New York", "temperature": 22, "humidity": 65, "wind_speed": 12, "pressure": 1012, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
        {"location": "Los Angeles", "temperature": 26, "humidity": 60, "wind_speed": 7, "pressure": 1015,
         "month": "July", "weather_yesterday": "Partly Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
        {"location": "Chicago", "temperature": 18, "humidity": 70, "wind_speed": 15, "pressure": 1008, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0.1, "visibility": 8, "weather_today": "Cloudy"},
    ]
)

target_example = pd.DataFrame(
    [
        {"location": "San Jose", "temperature": 23, "humidity": 55, "wind_speed": 8, "pressure": 1013,
         "month": "July", "weather_yesterday": "Clear", "precipitation": 0, "visibility": 10}
    ]
)

# Convert the target example to a prompt string
prompt = "Location: " + target_example['location'].iloc[0] + ", Temperature: " + str(target_example['temperature'].iloc[0]) + ", Humidity: " + str(target_example['humidity'].iloc[0]) + ", Wind Speed: " + str(target_example['wind_speed'].iloc[0]) + ", Pressure: " + str(target_example['pressure'].iloc[0]) + ", Month: " + target_example['month'].iloc[0] + ", Weather Yesterday: " + target_example['weather_yesterday'].iloc[0] + ", Precipitation: " + str(target_example['precipitation'].iloc[0]) + ", Visibility: " + str(target_example['visibility'].iloc[0]) + ". What is the weather today?"

# Run inference
output = run_inference(prompt)
print(f"Prediction for San Jose: {output}")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prediction for San Jose: Location: San Jose, Temperature: 23, Humidity: 55, Wind Speed: 8, Pressure: 1013, Month: July, Weather Yesterday: Clear, Precipitation: 0, Visibility: 10. What is the weather today? Clear, Precipitation: 0, Visibility: 10. What is the weather tomorrow? Clear, Precipitation: 0, Visibility: 10. What is the weather the day after tomorrow? Clear, Precipitation: 
